In [16]:
import numpy as np

In [17]:
def read_real_data(file_path):
    """
    Reads real data from a file where each line contains a single float value.

    Args:
        file_path (str): Path to the file containing real data.

    Returns:
        list[float]: List of real numbers read from the file.
    """
    real_data = []
    with open(file_path, "r") as file:
        for line in file:
            real_data.append(float(line.strip()))  # Convert each line to a float
    return real_data


def read_complex_data(file_path):
    """
    Reads complex data from a file where each line contains a real and imaginary part
    separated by a space.

    Args:
        file_path (str): Path to the file containing complex data.

    Returns:
        list[complex]: List of complex numbers read from the file.
    """
    complex_data = []
    with open(file_path, "r") as file:
        for line in file:
            parts = line.strip().split()  # Split the line into two parts
            real_part = float(parts[0])  # First part is the real component
            imag_part = float(parts[1])  # Second part is the imaginary component
            complex_data.append(complex(real_part, imag_part))  # Create a complex number
    return complex_data


In [32]:
real_file_1 = "./dataReal.txt"
complex_file_1 = "./dataComplex.txt"
real_file_2 = "./dataReal_tmp.txt"
complex_file_2 = "./dataComplex_tmp.txt"

real_data_1 = read_real_data(real_file_1)
real_data_2 = read_real_data(real_file_2)
complex_data_1 = read_complex_data(complex_file_1)
complex_data_2 = read_complex_data(complex_file_2)

In [35]:
cnt = 0
for i in range(len(real_data_1)):
    if real_data_1[i] != real_data_2[i]:
        cnt+=1
print(cnt)

cnt = 0
for i in range(len(complex_data_1)):
    if complex_data_1[i] != complex_data_2[i]:
        cnt+=1
print(cnt)

0
0


In [33]:
print(real_data_1[:3])
print(real_data_2[:3])

[1.0, -0.4257792915650727, -0.6374239897486895]
[1.0, -0.4257792915650727, -0.6374239897486895]


In [34]:
print(complex_data_1[:3])
print(complex_data_2[:3])

[(1+0j), (-0.4257792915650727+0.9048270524660195j), (-0.6374239897486895-0.7705132427757894j)]
[(1+0j), (-0.4257792915650727+0.9048270524660195j), (-0.6374239897486895-0.7705132427757894j)]


In [37]:
print(len(real_data_1), len(real_data_2))
print(len(complex_data_1), len(complex_data_2))

4096 4096
4096 4096


In [38]:
print(float(1))

1.0
